<a href="https://colab.research.google.com/github/anhle199/Demo-Stock-Price-Prediction/blob/master/predict_future_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Dense, LSTM
from keras.models import Sequential
import numpy as np
from pandas import DataFrame, RangeIndex, to_datetime
import yfinance as yf
from keras.models import load_model
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
import os

TRAIN_SIZE = 0.8
DEFAULT_MODEL_DATASET_PATH = '../model_dataset/'


def build_model_filename(crypto_name: str, path=DEFAULT_MODEL_DATASET_PATH) -> str:
    prefix = path if path.endswith('/') else path + '/'
    return f'{prefix}{crypto_name}_lstm_model.keras'


def download_live_cryptocurrency_data(name: str, currency: str, period: str, interval: str) -> DataFrame:
    data = yf.download(tickers=f"{name}-{currency}", period=period, interval=interval)
    data.insert(0, "Date", data.index)
    data.index = RangeIndex(0, len(data), 1)
    return data


def split_dataset(dataset, train_size=TRAIN_SIZE):
    index = int(len(dataset) * train_size)
    return dataset[:index], dataset[index:]


def prepare_dataset(crypto_name: str, period: str, interval: str) -> DataFrame:
    dataset = download_live_cryptocurrency_data(crypto_name, 'USD', period, interval)
    dataset['Date'] = to_datetime(dataset.Date, format='%Y-%m-%d')
    sorted_dataset = dataset.sort_values(by='Date', ascending=True, axis=0)
    filtered_dataset = DataFrame(data=sorted_dataset.Close.to_numpy(), index=sorted_dataset.Date, columns=['Close'])
    return filtered_dataset


def normalize_dataset(dataset: DataFrame, scaler):
    final_dataset = dataset.values
    train_data, _ = split_dataset(final_dataset)

    scaled_data = scaler.fit_transform(final_dataset)

    x_train_data, y_train_data = [], []
    for i in range(60, len(train_data)):
        x_train_data.append(scaled_data[i - 60 : i, 0])
        y_train_data.append(scaled_data[i, 0])

    x_train_data, y_train_data = np.array(x_train_data), np.array(y_train_data)
    x_train_data = np.reshape(x_train_data, (x_train_data.shape[0], x_train_data.shape[1], 1))

    return x_train_data, y_train_data


def train_lstm_model(x_train_data, y_train_data):
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train_data.shape[1], 1)))
    lstm_model.add(LSTM(units=50))
    lstm_model.add(Dense(1))

    lstm_model.compile(loss="mean_squared_error", optimizer="adam")
    lstm_model.fit(x_train_data, y_train_data, epochs=1, batch_size=1, verbose="2")

    return lstm_model


def get_sample_dataset(dataset: DataFrame, scaler):
    _, valid_data = split_dataset(dataset.values)
    inputs = dataset[len(dataset) - len(valid_data) - 60 :].values
    inputs = inputs.reshape(-1, 1)
    return scaler.transform(inputs)


def predict_close_price(model, scaler, inputs):
    X_test = []
    for i in range(60, inputs.shape[0]):
        X_test.append(inputs[i - 60 : i, 0])
    X_test = np.array(X_test)

    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    closing_price = model.predict(X_test)
    closing_price = scaler.inverse_transform(closing_price)

    return closing_price


# prepare dataset
dataset = prepare_dataset("BTC", '1d', '1m')
train_data, valid_data = split_dataset(dataset)

model_filename = build_model_filename("BTC")
scaler = MinMaxScaler(feature_range=(0, 1))

if os.path.isfile(model_filename):
    # use prebuilt model
    scaler.fit(train_data.values)
    model = load_model(model_filename)
else:
    # normalize dataset for training model
    train_dataset = prepare_dataset("BTC", '7d', '15m')
    x_train_data, y_train_data = normalize_dataset(train_dataset, scaler)

    # train lstm model
    model = train_lstm_model(x_train_data, y_train_data)
    model.save(model_filename)

# predict closing price
inputs = get_sample_dataset(dataset, scaler)
closing_price = predict_close_price(model, scaler, inputs)

valid_data['Predictions'] = closing_price